<a href="https://colab.research.google.com/github/americanas-data-platform/data-discovery-cidamo/blob/prototype/Discovery_B2W.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

Gráfico kaggle renderizado no backend. Como?

Extractor (classe? método?)
- entrada
  - Link do bucket
  - Link do arquivo
- saída
  - dataframe

transformer (classe? método?)
- entrada
  - dataframe
- saida
  - dicionário
 
Loader (classe? método?)
- entrada
  - 
- saida

## Extract

In [ ]:
url = "https://leofernanndes-datasets.s3.amazonaws.com/olist_products_dataset.csv"

def extract(url):
  """
  - encoding
  - separação 
  - tamanho do dataset
  - tipo de dados (imagem)?
  """
  try:
    dataframe = pd.read_csv(url)
    return dataframe

  except Exception as e:
    return "Não retorna dataframe"  


dataframe_retornado = extract(url)
dataframe_retornado

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0


## Transform

In [ ]:
"""
- Variavel quantitativa ou qualitativa;
- Contagem de nulos;
- Qualitativa:
  - moda
  - frequência
- Quantitativa:
  - Metricas do describe
- Variáveis não quali/quanti: Quais estatisticas usar?
  - ids e códigos?
  - datas?
"""

import pandas as pd
import numpy as np
import os
from pandas.api.types import is_numeric_dtype, is_string_dtype

class DataInfo:
    def __init__(self, url = None):
        # self.data = dataframe
        self.url = url
        
        
    def extractCSV(self):
        url = self.url
        
        try:
            arquivoCsv = pd.read_csv(url)
            self.data = arquivoCsv
            return arquivoCsv.head()

        except Exception as e:
            return "Não retorna dataframe"  
    
    def describeColumns(self):
        data = self.data
        df = pd.DataFrame(columns = ['nan_values', 'not_nan', 'uniques', 'type'])
        index = data.columns.values

        for i in range(len(index)):
            df.loc[i, 'nan_values'] = data[index[i]].isnull().sum()
            df.loc[i, 'not_nan'] = data[index[i]].notnull().sum()
            df.loc[i, 'uniques'] = len(data[index[i]].unique())
            df.loc[i, 'type'] = str(data[index[i]].dtype)

        df.index = index
        df.reset_index(inplace= True)
        df.rename(columns = {'index': 'columns_names'}, inplace= True)
        return df
    
    def describeNumbers(self):
        data = self.data
        df = pd.DataFrame(columns = ['mean', 'std', 'min', 'median', 'max'])
        index = data.columns.values
        columns_names = []
        for i in range(len(index)):

            if is_numeric_dtype(data[index[i]]):
                df.loc[i, 'mean'] = np.round(data[index[i]].mean(), 3)
                df.loc[i, 'std'] = np.round(data[index[i]].std(), 3)
                df.loc[i, 'min'] = np.round(data[index[i]].min(), 3)
                df.loc[i, 'median'] = np.round(data[index[i]].median(), 3)
                df.loc[i, 'max'] = np.round(data[index[i]].max(), 3)
                columns_names.append(f'{index[i]}')
            else:
                pass

        df.reset_index(inplace = True)
        df.rename(columns = {'index': 'columns_names'}, inplace=True)
        df['columns_names'] = columns_names
        return df

    def describeCat(self):
        data = self.data      
        df = pd.DataFrame(columns = ['unique', 'moda', 'top10', 'down10'])
        index = data.columns.values
        columns_names = []
        for i in range(len(index)):

            if is_string_dtype(data[index[i]]):

                try: 
                    tempo = pd.to_datetime(data[index[i]])
                    # E se for json?
                except: 

                    df.loc[i, 'unique'] = data[index[i]].unique().tolist()
                    df.loc[i, 'moda'] = data[index[i]].mode().tolist()
                    df.loc[i, 'top10'] = data[index[i]].value_counts().head(10).index.tolist()
                    df.loc[i, 'down10'] = data[index[i]].value_counts().tail(10).index.tolist()
                    columns_names.append(f'{index[i]}')

            else:
                pass

        df.reset_index(inplace = True)
        df.rename(columns = {'index': 'columns_names'}, inplace=True)
        df['columns_names'] = columns_names
        return df

In [ ]:
c = DataInfo(url)

In [ ]:
c.extractCSV()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [ ]:
c.describeColumns()

,columns_names,nan_values,not_nan,uniques,type
0,product_id,0,32951,32951,object
1,product_category_name,610,32341,74,object
2,product_name_lenght,610,32341,67,float64
3,product_description_lenght,610,32341,2961,float64
4,product_photos_qty,610,32341,20,float64
5,product_weight_g,2,32949,2205,float64
6,product_length_cm,2,32949,100,float64
7,product_height_cm,2,32949,103,float64
8,product_width_cm,2,32949,96,float64


In [ ]:
c.describeNumbers()

,columns_names,mean,std,min,median,max
0,product_name_lenght,48.477,10.246,5,51,76
1,product_description_lenght,771.495,635.115,4,595,3992
2,product_photos_qty,2.189,1.737,1,1,20
3,product_weight_g,2276.47,4282.04,0,700,40425
4,product_length_cm,30.815,16.914,7,25,105
5,product_height_cm,16.938,13.638,2,13,105
6,product_width_cm,23.197,12.079,6,20,118


In [ ]:
c.describeCat()

,columns_names,unique,moda,top10,down10
0,product_id,"[1e9e8ef04dbcff4541ed26657ea517e5, 3aa071139cb...","[00066f42aeeb9f3007548bb9d3f33c38, 00088930e92...","[c615ac0ef96d723029051d29d7cd2d16, 01586bfbe1d...","[b1710c76a2b88d24f13cf3afbb18d700, c3779d48220..."
1,product_category_name,"[perfumaria, artes, esporte_lazer, bebes, util...",[cama_mesa_banho],"[cama_mesa_banho, esporte_lazer, moveis_decora...","[fraldas_higiene, portateis_cozinha_e_preparad..."


## Load